In [ ]:
import numpy as np
import random
from math import sqrt, log

In [ ]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
ITERATIONS = 1500
EMPTY_SPACE = 0

'''Player1 = 1
Player2 = 2
node = None'''

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

'Player1 = 1\nPlayer2 = 2\nnode = None'

In [ ]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(_board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(_board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(_board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(_board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(_board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )
def eval_terminal(board,hero, villain):
    if four_in_a_row(board,hero):
        return 1
    elif four_in_a_row(board,villain):
        return -1
    else:
        return 0

'''def minimax(state, max_depth, is_player_minimizer):
    if max_depth == 0 or state.is_end_state():
        # We're at the end. Time to evaluate the state we're in
        return evaluation_function(state)

    # Is the current player the minimizer?
    if is_player_minimizer:
        value = -math.inf
        for move in state.possible_moves():
            evaluation = minimax(move, max_depth - 1, False)
            min = min(value, evaluation)
        return value

    # Or the maximizer?
    value = math.inf
    for move in state.possible_moves():
        evaluation = minimax(move, max_depth - 1, True)
        max = max(value, evaluation)
    return value'''

def minmax(board,hero, villain,iterations):
    if iterations == 0:
        return None,0
    val = eval_terminal(board,hero, villain)
    possible = valid_moves(board)
    if val != 0 or not possible:
        return None, val
    evaluations = list()
    for ply in possible:
        play(board, ply, hero)
        _, val = minmax(board,villain,hero,iterations-1)
        take_back(board, ply)
        evaluations.append((ply, -val))
        if val==-1:
            break
        
    return max(evaluations, key=lambda k: k[1])
    

In [ ]:

class Board(object):
  def __init__(self):
    self.__last_row = 0
    self.__last_column = 0
    self.board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
    
  def make_copy(self):
    b = Board()
    for row in range(NUM_COLUMNS):
      for col in range(COLUMN_HEIGHT):
        b.board[row][col] = self.board[row][col]
    return b

  def easy_copy(self):
    b = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
    for row in range(NUM_COLUMNS):
      for col in range(COLUMN_HEIGHT):
        b[row][col] = self.board[row][col]
    return b
    
  def print_board(self):
    for row in range(len(self.board)):
      #for space in self.board[row]:
      print(self.board[row]) 
      #print( )
    print( '-' * 50)

  def add_piece(self, piece, column):
    if self.__column_filled(column):
      return False

    for row in range(1, len(self.board)):
      if self.board[row][column] != EMPTY_SPACE:
        self.board[row - 1][column] = piece
        self.__last_row = row - 1
        self.__last_column = column
        return True

    # If this is the first piece in the column
    self.board[-1][column] = piece
    self.__last_row = len(self.board) - 1
    self.__last_column = column
    return True

  @property
  def coordinate_of_most_recent_piece(self):
    return (self.__last_row, self.__last_column)

  @property
  def last_added_piece(self):
    return self.board[self.__last_row][self.__last_column]

  """
  This function gives the weights to wins(1.0), losses(0.0), and draws(0.5)
  """
  def get_result_for_player(self, player_piece):
    if self.is_draw():
      return 0.5
    elif self.last_added_piece == player_piece:
      return 1.0
    else:
      return 0.0

  def is_draw(self):
    not self.winner_found and self.spaces_left == 0

  def winner_found(self, player):
    if self.four_in_a_row(player):
      return True
    return False
    '''row, column = self.coordinate_of_most_recent_piece
    return self.vertical_winner(column) or self.horizontal_winner(row) or self.diagonal_winner(row, column)'''

  def possible_moves(self, player):
    if self.winner_found(player):
      return []
    return   self.valid_moves()#[i for i in range(COLUMN_HEIGHT) if not self.__column_filled(i)]

  def spaces_left(self):
    count = 0
    for row in self.board:
      for space in row:
        if space == EMPTY_SPACE:
          count += 1
    return count

  def __column_filled(self, col):
    for row in self.board:
      if row[col] == EMPTY_SPACE:
        return False
    return True

  def four_in_a_row(self, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(self.board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(self.board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(self.board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(self.board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )
  def valid_moves(self):
    """Returns columns where a disc may be played"""
    return [n for n in range(COLUMN_HEIGHT) if self.board[0, n] == 0]


In [ ]:
class Node:
  def __init__(self, state, piece, column=None, parent=None):
    self.column = column
    self.parentNode = parent
    self.current_player_piece = piece
    self.untried_moves = state.possible_moves(int(piece))
    self.children = list()
    self.wins = 0
    self.visits = 0

  def uct_select_child(self):
    max_score, max_child = 0, None
    for child in self.children:
      score = child.wins / child.visits + sqrt(2*log(self.visits) / child.visits)
      if score > max_score:
        max_child = child
        max_score = score
    return max_child

  def add_child(self, col, state):
    node = Node(state.make_copy(), self.get_next_piece(self.current_player_piece), column=col, parent=self)
    self.untried_moves.remove(col)
    self.children.append(node)
    return node

  def update(self, result):
    self.visits += 1
    self.wins += result

  def get_next_piece(self, piece):
    if piece == '1':
      return '2'
    return '1'

In [ ]:
class MonteCarlo(object):
  def __init__(self, state, piece, iterations, last_node=None):
    if last_node is not None:
      self.root = last_node
    else:
      self.root = Node(state.make_copy(), piece)
    self.original_state = state
    self.iterations = iterations

  def get_move(self):
    for _ in range(self.iterations):
      node = self.root
      state = self.original_state.make_copy()
      node = self.select(node, state)     
      node = self.expand(node, state)
      state = self.rollout(state, node)     
      self.backpropagate(node, state)
    return self.root, sorted(self.root.children, key=lambda c: c.wins/c.visits)[-1].column

  def select(self, node, state):
    while len(node.untried_moves) == 0 and len(node.children) != 0:
      node = node.uct_select_child()
      state.add_piece(node.current_player_piece, node.column)
    return node

  def expand(self, node, state):
    if len(node.untried_moves) != 0:
      col = random.choice(node.untried_moves)
      state.add_piece(node.current_player_piece, col)
      node = node.add_child(col, state)
    return node

  def rollout(self, state, node=None):
    while len(state.possible_moves(int(node.current_player_piece))) != 0:
      column = random.choice(state.possible_moves(int(node.current_player_piece)))
      piece = self.__get_next_piece(state.last_added_piece)
      state.add_piece(piece, column)
    return state

  def backpropagate(self, node, state):
    while node is not None:
      node.update(state.get_result_for_player(node.current_player_piece))
      node = node.parentNode

  def __get_next_piece(self, piece):
    if piece == '1':
      return '2'
    return '1'

In [ ]:
def __advance_turn(players,i):
    return players[i%2]

def __navigate_to_node_for_move(node, column, board):
    for child in node.children:
      if child.column == column:
        return child
    return Node(board, node.current_player_piece)

In [52]:
players =  ['1','2'] 
board = Board()
current_player = None

i = 0
i = i + 1
current_player = __advance_turn(players,i)
node = Node(board, current_player)
#print (self.current_player.name, ' goes first')
board.print_board()
while True:
      if i % 2 != 0:
        #column = self.get_move()
        player = 2
        msg = 'Which column number would you like to play? '
        column = input(msg)
        #node, column = MonteCarlo(board.make_copy(), players[1], ITERATIONS, last_node=node).get_move()
        print ('You have choosen column', column)
        board.add_piece(players[1], int(column))
        '''while not self.board.add_piece(self.current_player.piece, column):
          print ('That is not a valid move. Please select a different column')
          column = self.get_move()'''

      else:
        player = 1
        node, column = MonteCarlo(board.make_copy(), players[0], ITERATIONS, last_node=node).get_move()
        print ('Computer chooses column', column)
        board.add_piece(players[0], column)

      board.print_board()
      print("updated")
      node = __navigate_to_node_for_move(node, column, board)
      if four_in_a_row( board.easy_copy(), player): #self.board.winner_found()
        print ('***** ' +' Player '+ current_player + ' wins!')
        break

      if board.spaces_left() == 0:
        print ('***** Tie game')
        break
      i = i + 1
      current_player = __advance_turn(players,i)


[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
--------------------------------------------------
You have choosen column 1
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 2 0 0 0 0]
--------------------------------------------------
updated
Computer chooses column 2
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 2 1 0 0 0]
--------------------------------------------------
updated
You have choosen column 1
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 2 1 0 0 0]
--------------------------------------------------
updated
Computer chooses column 2
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 2 1 0 0 0]
[0 2 1 0 0 0]
--------------------------------------------------
updated
You have choosen column 1
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 2 0 0 0 0]
[0 2 1 0 0 0]
[0 2 1 0 

In [ ]:
'''play(BoardObject.board,0,Player1)
play(BoardObject.board,0,Player1)
play(BoardObject.board,0,Player1)
play(BoardObject.board,1,Player2)
print(BoardObject.board)
print( )
if four_in_a_row(BoardObject.board,Player1):
    print("yaaay")


best_ply, eval = minmax(board,Player1,Player2, ITERATIONS)
play(board,best_ply,Player1)
print(board)
print( )

best_ply, eval = minmax(board,Player2,Player1, ITERATIONS)
play(board,best_ply,2)
print(board)
print( )
node, column = MonteCarlo(BoardObject.make_copy(), Player1, ITERATIONS).get_move()
print(column)
play(BoardObject.board,column,Player1)
while 1:

    print(BoardObject.board)
    print( )
    node, column = MonteCarlo(BoardObject.make_copy(), Player2, ITERATIONS, last_node=node).get_move()
    #print(column)
    if(column == -1):
        break
    play(BoardObject.board,column,Player2)

    print( )
    node, column = MonteCarlo(BoardObject.make_copy(), Player1, ITERATIONS, last_node=node).get_move()
    if(column == -1):
        break
    #print(column)
    play(BoardObject.board,column,Player1)
    
print(BoardObject.board)
#while 1 :
 #   print()'''

'play(BoardObject.board,0,Player1)\nplay(BoardObject.board,0,Player1)\nplay(BoardObject.board,0,Player1)\nplay(BoardObject.board,1,Player2)\nprint(BoardObject.board)\nprint( )\nif four_in_a_row(BoardObject.board,Player1):\n    print("yaaay")\n\n\nbest_ply, eval = minmax(board,Player1,Player2, ITERATIONS)\nplay(board,best_ply,Player1)\nprint(board)\nprint( )\n\nbest_ply, eval = minmax(board,Player2,Player1, ITERATIONS)\nplay(board,best_ply,2)\nprint(board)\nprint( )\nnode, column = MonteCarlo(BoardObject.make_copy(), Player1, ITERATIONS).get_move()\nprint(column)\nplay(BoardObject.board,column,Player1)\nwhile 1:\n\n    print(BoardObject.board)\n    print( )\n    node, column = MonteCarlo(BoardObject.make_copy(), Player2, ITERATIONS, last_node=node).get_move()\n    #print(column)\n    if(column == -1):\n        break\n    play(BoardObject.board,column,Player2)\n\n    print( )\n    node, column = MonteCarlo(BoardObject.make_copy(), Player1, ITERATIONS, last_node=node).get_move()\n    if(c